In [1]:

from sklearn.datasets import fetch_california_housing
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import statsmodels.api as sm
import nbformat as nbf
import sympy as sp
from sympy import symbols, Matrix, diff
import random


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

from sklearn.preprocessing import StandardScaler
from matplotlib.animation import FuncAnimation
from sklearn.metrics import mean_squared_error, r2_score
import pickle
from prettytable import PrettyTable




In [2]:
df = pd.read_pickle("preprocessed_df.pkl")  # or read_csv
print(df.info(memory_usage="deep"))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   HouseAge                 20640 non-null  float64
 1   Medinc_log               20640 non-null  float64
 2   AveRooms_boxcox          20640 non-null  float64
 3   AveBedrms_boxcox         20640 non-null  float64
 4   Population_log           20640 non-null  float64
 5   AveOccup_boxcox          20640 non-null  float64
 6   spatial_proximity_score  20640 non-null  float64
 7   MedHouseVal              20640 non-null  float64
dtypes: float64(8)
memory usage: 1.3 MB
None


## Task - 1 : Formulating this as LASSO, and specify the Objective Function and Convex but not Differentiable

### Formulating this as Lasso : 

We need to predict the target from features X  : 

![image27](Equations/27.png)


Now add the penalty term, such that it reduces the weights of the Irrelevant features to 0


![image28](Equations/28.png)

The Final **Objective Function** Would be : 

![image29](Equations/29.png)






#### Why This Is a Convex Problem
- Squared loss is a **smooth convex** curve (bowl shape).
- L1 norm is a **convex** V-shape.
- Sum of two convex functions is **always convex**.

So the whole LASSO problem is **convex**.


####  Why It Is Non-Differentiable
- L1 norm has a **sharp corner at 0**.
- At this point, derivative does not exist.
- That’s why LASSO is **convex but not differentiable**.


#### Why L2 (Ridge) Is Not Enough
- L2 penalty shrinks all weights smoothly.
- It **never makes coefficients exactly zero**.
- No feature selection happens.
- Still sensitive to outliers.


#### Why L1 Is Used
- L1 can make weights **exactly zero**.
- Gives **feature selection** automatically.
- Keeps model simple.
- Works well when many features exist.


#### Solutions For this Problem :
We use:
- **Subgradient methods**
- **Proximal gradient methods (Soft thresholding)**

These methods handle the sharp corner of L1.


## Task - 2 : Derive soft thresholding used in ISTA with update step

#### Lasso defined as : 

![image29](Equations/29.png)

![image30](Equations/30.png)

![image31](Equations/31.png)

Goal : Solve for **(w*)**

![image32](Equations/32.png)


Proximal Operator is Defined as follows : 


![image33.png](Equations/33.png)


**Idea of ISTA(Iterative Shrinkage thresholding Algorithm):**

![image34](Equations/34.png)

![image35](Equations/35.png)

So, We need the proximity for L1, so we reduce it to the 1D as follows : 

![image36](Equations/36.png)

So, the Solution for this would be : 

![image37](Equations/37.png)


### Final Thresholding Proximity Formula Will be : 



![image38](Equations/38.png)

And the Compact Form would be : 

![image39](Equations/39.png)


**ISTA Update Would be**

![image40](Equations/40.png)

![image41](Equations/41.png)


**Safe Step Size Rule Would be**


![image42](Equations/42.png)

![image43](Equations/43.png)





### Task - 3 : Implement the Lasso with ISTA with λ = [0.01, 0.1, 1]

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   HouseAge                 20640 non-null  float64
 1   Medinc_log               20640 non-null  float64
 2   AveRooms_boxcox          20640 non-null  float64
 3   AveBedrms_boxcox         20640 non-null  float64
 4   Population_log           20640 non-null  float64
 5   AveOccup_boxcox          20640 non-null  float64
 6   spatial_proximity_score  20640 non-null  float64
 7   MedHouseVal              20640 non-null  float64
dtypes: float64(8)
memory usage: 1.3 MB


In [4]:

df = df.copy()

X = df.drop(columns=['MedHouseVal']).values
y = df['MedHouseVal'].values.reshape(-1, 1)

# Normalize features
X_mean = X.mean(axis=0)
X_std = X.std(axis=0) + 1e-8
X = (X - X_mean) / X_std

n, d = X.shape

# Gradient Descent for Ridge Regression
def ridge_gradient_descent(X, y, lam, lr=0.01, iters=500):
    n, d = X.shape
    w = np.zeros((d,1))

    for _ in range(iters):
        grad = (1/n) * (X.T @ (X@w - y)) + lam * w
        w -= lr * grad
    return w

# Soft Thresholding

def soft_thresholding(z, alpha):
    return np.sign(z) * np.maximum(np.abs(z) - alpha, 0)

# ISTA Updation

def lasso_ista(X, y, lam, lr=0.01, iters=1000):
    n, d = X.shape
    w = np.zeros((d,1))

    for _ in range(iters):
        grad = (1/n) * (X.T @ (X@w - y))
        w = soft_thresholding(w - lr*grad, lr*lam)
    return w


lambdas = [0.01, 0.1, 1, 0.005, 0.05, 0.5, 2e-2, 2e-1, 5e-2, 5e-1, 2e-3, 2e-4, 5e-3, 5e-4]

import pandas as pd

results = {}

for lam in lambdas:
    print(f"\n========== λ = {lam} ==========")
    
    w_ridge = ridge_gradient_descent(X, y, lam)
    w_lasso = lasso_ista(X, y, lam)

    results[lam] = {"ridge": w_ridge, "lasso": w_lasso}

    table = pd.DataFrame({
        "Feature": df.drop(columns=['MedHouseVal']).columns,
        "Ridge Weight": w_ridge.flatten(),
        "LASSO Weight": w_lasso.flatten()
    })

    print(table.to_string(index=False))




========== λ = 0.01 ==========
                Feature  Ridge Weight  LASSO Weight
               HouseAge      0.113594      0.111920
             Medinc_log      0.758464      0.829227
        AveRooms_boxcox     -0.034653     -0.109466
       AveBedrms_boxcox      0.038826      0.075311
         Population_log      0.028671      0.004826
        AveOccup_boxcox     -0.290120     -0.268938
spatial_proximity_score      0.288159      0.256074

========== λ = 0.1 ==========
                Feature  Ridge Weight  LASSO Weight
               HouseAge      0.104358      0.024214
             Medinc_log      0.686386      0.660326
        AveRooms_boxcox     -0.004207     -0.000000
       AveBedrms_boxcox      0.021063      0.000000
         Population_log      0.024802     -0.000000
        AveOccup_boxcox     -0.269410     -0.194263
spatial_proximity_score      0.274062      0.226851

========== λ = 1 ==========
                Feature  Ridge Weight  LASSO Weight
               HouseAge 

### Task - 4 : Convergence of Training plot, MSE and R2, Sparsity Ratio

In [5]:
# For λ = 0.1 Convergence plot for LASSO training objective Vs iterations using plotly

lam = 0.1
n, d = X.shape
w = np.zeros((d,1))
iters = 1000
objectives = []
for it in range(iters):
    grad = (1/n) * (X.T @ (X@w - y))
    w = soft_thresholding(w - 0.01*grad, 0.01*lam)
    
    residual = y - X @ w
    lasso_obj = (1/(2*n)) * np.sum(residual**2) + lam * np.sum(np.abs(w))
    objectives.append(lasso_obj)

fig = px.line(x=list(range(iters)), y=objectives, labels={'x':'Iterations', 'y':'LASSO Objective'}, 
              title='LASSO Training Objective vs Iterations (λ=0.1)')
fig.show()


In [6]:
# For λ = 0.01 Convergence plot for LASSO training objective Vs iterations using plotly

lam = 0.01
n, d = X.shape
w = np.zeros((d,1))
iters = 1000
objectives = []
for it in range(iters):
    grad = (1/n) * (X.T @ (X@w - y))
    w = soft_thresholding(w - 0.01*grad, 0.01*lam)
    
    residual = y - X @ w
    lasso_obj = (1/(2*n)) * np.sum(residual**2) + lam * np.sum(np.abs(w))
    objectives.append(lasso_obj)
fig = px.line(x=list(range(iters)), y=objectives, labels={'x':'Iterations', 'y':'LASSO Objective'}, 
              title='LASSO Training Objective vs Iterations (λ=0.01)')
fig.show()



In [7]:
# For λ = 1 Convergence plot for LASSO training objective Vs iterations using plotly

lam = 1
n, d = X.shape
w = np.zeros((d,1))
iters = 1000
objectives = []
for it in range(iters):
    grad = (1/n) * (X.T @ (X@w - y))
    w = soft_thresholding(w - 0.01*grad, 0.01*lam)
    
    residual = y - X @ w
    lasso_obj = (1/(2*n)) * np.sum(residual**2) + lam * np.sum(np.abs(w))
    objectives.append(lasso_obj)
fig = px.line(x=list(range(iters)), y=objectives, labels={'x':'Iterations', 'y':'LASSO Objective'}, 
              title='LASSO Training Objective vs Iterations (λ=1)')
fig.show()



In [8]:
# Report MSE, R2 and Sparsity Ratio for different λ values in list for LASSO in Pretty Table format

table = PrettyTable()
table.field_names = ["λ", "MSE", "R2 Score", "Sparsity Ratio"]
for lam in lambdas:
    w_lasso = results[lam]["lasso"]
    y_pred = X @ w_lasso

    mse = mean_squared_error(y, y_pred)
    r2 = r2_score(y, y_pred)
    sparsity_ratio = np.sum(w_lasso == 0) / len(w_lasso)

    table.add_row([lam, round(mse, 4), round(r2, 4), round(sparsity_ratio, 4)])
print(table)




+--------+--------+----------+----------------+
|   λ    |  MSE   | R2 Score | Sparsity Ratio |
+--------+--------+----------+----------------+
|  0.01  | 4.7935 | -2.5999  |      0.0       |
|  0.1   | 4.8432 | -2.6372  |     0.4286     |
|   1    | 5.6105 | -3.2135  |      1.0       |
| 0.005  | 4.792  | -2.5988  |      0.0       |
|  0.05  | 4.8166 | -2.6173  |     0.4286     |
|  0.5   | 5.2629 | -2.9525  |     0.8571     |
|  0.02  | 4.7977 | -2.6031  |     0.1429     |
|  0.2   | 4.9335 | -2.7051  |     0.5714     |
|  0.05  | 4.8166 | -2.6173  |     0.4286     |
|  0.5   | 5.2629 | -2.9525  |     0.8571     |
| 0.002  | 4.7914 | -2.5983  |      0.0       |
| 0.0002 | 4.7911 | -2.5981  |      0.0       |
| 0.005  | 4.792  | -2.5988  |      0.0       |
| 0.0005 | 4.7911 | -2.5981  |      0.0       |
+--------+--------+----------+----------------+


In [11]:
tol = 1e-4
min_iter = None

for it in range(iters):
    grad = (1/n) * (X.T @ (X@w - y))
    w = soft_thresholding(w - 0.01*grad, 0.01*lam)

    residual = y - X @ w
    lasso_obj = (1/(2*n)) * np.sum(residual**2) + lam * np.sum(np.abs(w))
    objectives.append(lasso_obj)

    # convergence check
    if it > 0 and min_iter is None:
        if abs(objectives[-1] - objectives[-2]) < tol:
            min_iter = it

# If never satisfied, assign last iteration
if min_iter is None:
    min_iter = it

print("Minimal convergence iteration =", min_iter)
print("Minimal LASSO objective =", objectives[min_iter])


Minimal convergence iteration = 1
Minimal LASSO objective = 2.8052415994936264


### Very bad Move, as R score is -ve which means underfitting, i.e., the data might not be sufficient, so we are introducing the new features 

In [ ]:
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X)   # X is your normalized original data

print("Original shape:", X.shape)
print("Polynomial shape:", X_poly.shape)


In [ ]:
X_mean = X_poly.mean(axis=0)
X_std  = X_poly.std(axis=0) + 1e-8
X_poly = (X_poly - X_mean) / X_std


In [ ]:
w_lasso   = lasso_ista(X_poly, y, lam)

def predict(X, w):
    return X @ w


def mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def r2_score_custom(y_true, y_pred):
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1 - (ss_res / ss_tot)

y_pred_poly = predict(X_poly, w_lasso)
mse_poly = mse(y, y_pred_poly)
r2_poly = r2_score_custom(y, y_pred_poly)
print(f"Polynomial Features LASSO Regression - MSE: {mse_poly:.4f}, R2 Score: {r2_poly:.4f}")



# <span style="color:#2E86C1;">Final Report</span>



## <span style="color:#28B463;">Model Setup</span>

- Model: LASSO Regression  
- Feature Engineering: Polynomial Features (degree = 2)  
- Objective: Evaluate whether polynomial expansion improves predictive performance.



## <span style="color:#CA6F1E;">Model Metrics</span>

| <span style="color:#AF7AC5;">Metric</span> | <span style="color:#AF7AC5;">Value</span> |
|--------|--------|
| **MSE** | **5.6105** |
| **R² Score** | **–3.2135** |



## <span style="color:#CD6155;">Interpretation</span>

- R² is strongly negative → model performs worse than predicting the mean.  
- MSE increased → no improvement after polynomial feature expansion.  
- LASSO zeroed out most polynomial terms → high sparsity → strong underfitting.



### <span style="color:#E74C3C;">❌ This model is not suitable for this dataset.</span>

Polynomial Features + LASSO does **not** improve accuracy and makes the model perform significantly worse.
